In [ ]:
%pip install pandas prophet boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 161.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 114.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from prophet import Prophet
import boto3
import os
import json


# 1. Télécharger le fichier depuis S3
local_file = "your-s3-bucket" 

# Charger les données dans un DataFrame Pandas
data = pd.read_csv(local_file)
s3= boto3.client('s3')
data

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:279: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


,STN Code,Dissolved Oxygen,Location Name,State Name,Year,pH,Conductivity,BOD,Nitrate N + Nitrite N,Fecal Coliform,Total Coliform,WQI,lat,lon
0,1790,6.00,PULICATE LAKE,ANDHRA PRADESH,2017,7.80,79935.0,1.65,3.775,2.0,1200.0,88.277923,13.695333,80.059293
1,2353,6.35,KONDACHARLA-AAVA LAKE,ANDHRA PRADESH,2017,7.65,815.5,1.80,2.260,20.0,1375.0,68.080478,17.603203,83.000845
2,2205,4.70,MER BEEL AT MADHABPUR,ASSAM,2017,6.35,89.0,8.60,0.900,1150.0,3380.0,138.790236,26.407384,93.255130
3,2206,5.55,DALONI BEEL NEAR JOGIGHOPA,ASSAM,2017,7.20,103.5,1.85,0.750,750.0,2800.0,80.709022,26.407384,93.255130
4,2557,6.70,KAWAR LAKE,BIHAR,2017,7.55,273.5,2.45,0.000,16500.0,26500.0,440.315367,25.512541,86.088339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1282,2544,9.60,SAHEBBANDH AT PURULIA,WEST BENGAL,2022,7.40,283.0,3.55,0.680,2900.0,6550.0,128.186851,22.996495,87.685588
1283,2522,8.00,SINCHAL LAKE FOR DARJEELING,WEST BENGAL,2022,7.40,31.5,1.40,0.715,116.0,820.0,53.921655,22.996495,87.685588
1284,5163,7.35,SUBHAS SAROBAR,WEST BENGAL,2022,7.75,387.5,3.75,0.830,1500.0,2400.0,114.995558,22.996495,87.685588
1285,2521,7.75,WATER RESERVIOR AT DELO LAKE,WEST BENGAL,2022,7.35,27.0,1.10,0.735,1685.0,5570.0,87.375709,22.996495,87.685588


In [ ]:
def forecast_parameters_sagemaker(data, start_year=2023, end_year=2024):
    import os
    from collections import defaultdict
    import json
    from datetime import datetime
    from prophet import Prophet  # Assuming Prophet is properly installed
    import numpy as np
    
    data['Year'] = pd.to_datetime(data['Year'], format='%Y')

    # Initialiser une structure de données pour collecter les résultats
    final_results = defaultdict(lambda: defaultdict(dict))

    for stn_code in data['STN Code'].unique():
        stn_data = data[data['STN Code'] == stn_code]

        if stn_data.empty:
            continue

        # Collecter les informations générales de la station
        stn_info = {
            "Location Name": stn_data.iloc[0].get("Location Name"),
            "State Name": stn_data.iloc[0].get("State Name"),
            "lat": stn_data.iloc[0].get("lat"),
            "lon": stn_data.iloc[0].get("lon"),
            "processed_timestamp": datetime.now().isoformat()
        }
        parameters = {}
        for column in ['WQI', 'Dissolved Oxygen', 'pH', 'Conductivity', 'BOD', 
                       'Nitrate N + Nitrite N', 'Fecal Coliform', 'Total Coliform']:
            # Préparer les données pour Prophet
            stn_data_param = stn_data.rename(columns={'Year': 'ds', column: 'y'})
            stn_data_param = stn_data_param[['ds', 'y']].dropna()

            if len(stn_data_param) >= 2:
                model = Prophet(changepoint_prior_scale=0.001, yearly_seasonality=True)
                model.fit(stn_data_param)

                # Prédire les valeurs futures
                future = model.make_future_dataframe(periods=(end_year - datetime.now().year + 1), freq='Y')
                forecast = model.predict(future)
                forecast = forecast[(forecast['ds'].dt.year >= start_year) & 
                                    (forecast['ds'].dt.year <= end_year)]

                forecast_values = forecast[['ds', 'yhat']].set_index('ds').to_dict()['yhat']
                
                for k, v in forecast_values.items():
                    
                    # Extract previous values from stn_data_param
                    previous_values = stn_data_param['y']

                    # Generate a random value based on previous data
                    if v < 0:
                        random_val = np.random.uniform(
                            low=max(0, previous_values.min() * 0.7),  # Ensure positivity
                            high=previous_values.mean() * 1.1        # Slightly above the mean
                        )
                        v = max(v, random_val)  # Use the larger value to minimize under-prediction
            
                    year = k.year
                    final_results[year][stn_code]["Location Name"] = stn_info["Location Name"]
                    final_results[year][stn_code]["State Name"] = stn_info["State Name"]
                    final_results[year][stn_code]["lat"] = stn_info["lat"]
                    final_results[year][stn_code]["lon"] = stn_info["lon"]
                    final_results[year][stn_code][column] = v

    # Upload each station's forecast as JSON to S3 in the required hierarchy
    for year, stations in final_results.items():
        for stn_code, station_data in stations.items():
            # Convert station data to JSON
            station_data["Fecal Coliform"] = int(station_data["Fecal Coliform"])
            station_data["Total Coliform"] = int(station_data["Total Coliform"])
            station_data["processed_timestamp"] = datetime.now().isoformat()
            file_content = json.dumps(station_data)
            folder_path = f"water_data/year={year}/stn_code={stn_code}"
            file_name = f"{folder_path}/forecast_{year}_{stn_code}.json"

            # Upload to S3
            s3.put_object(
                Body=file_content.encode('utf-8'),
                Bucket="water-data-03042001",
                Key=file_name
            )
            print(f"Uploaded: {file_name}")

    return final_results

fin = forecast_parameters_sagemaker(data, start_year=2023, end_year=2024)

15:49:42 - cmdstanpy - INFO - Chain [1] start processing
15:49:42 - cmdstanpy - INFO - Chain [1] done processing
15:49:42 - cmdstanpy - INFO - Chain [1] start processing
15:49:42 - cmdstanpy - INFO - Chain [1] done processing
15:49:42 - cmdstanpy - INFO - Chain [1] start processing
15:49:42 - cmdstanpy - INFO - Chain [1] done processing
15:49:42 - cmdstanpy - INFO - Chain [1] start processing
15:49:43 - cmdstanpy - INFO - Chain [1] done processing
15:49:43 - cmdstanpy - INFO - Chain [1] start processing
15:49:43 - cmdstanpy - INFO - Chain [1] done processing
15:49:43 - cmdstanpy - INFO - Chain [1] start processing
15:49:43 - cmdstanpy - INFO - Chain [1] done processing
15:49:43 - cmdstanpy - INFO - Chain [1] start processing
15:49:43 - cmdstanpy - INFO - Chain [1] done processing
15:49:43 - cmdstanpy - INFO - Chain [1] start processing
15:49:43 - cmdstanpy - INFO - Chain [1] done processing
15:49:44 - cmdstanpy - INFO - Chain [1] start processing
15:49:44 - cmdstanpy - INFO - Chain [1]

Uploaded: water_data/year=2023/stn_code=1790/forecast_2023_1790.json
Uploaded: water_data/year=2023/stn_code=2353/forecast_2023_2353.json
Uploaded: water_data/year=2023/stn_code=2205/forecast_2023_2205.json
Uploaded: water_data/year=2023/stn_code=2206/forecast_2023_2206.json
Uploaded: water_data/year=2023/stn_code=2557/forecast_2023_2557.json
Uploaded: water_data/year=2023/stn_code=2571/forecast_2023_2571.json
Uploaded: water_data/year=2023/stn_code=1549/forecast_2023_1549.json
Uploaded: water_data/year=2023/stn_code=2269/forecast_2023_2269.json
Uploaded: water_data/year=2023/stn_code=3175/forecast_2023_3175.json
Uploaded: water_data/year=2023/stn_code=3176/forecast_2023_3176.json
Uploaded: water_data/year=2023/stn_code=3177/forecast_2023_3177.json
Uploaded: water_data/year=2023/stn_code=3178/forecast_2023_3178.json
Uploaded: water_data/year=2023/stn_code=3179/forecast_2023_3179.json
Uploaded: water_data/year=2023/stn_code=3180/forecast_2023_3180.json
Uploaded: water_data/year=2023/stn